# Introduction

AirBnB is a marketplace for short term rentals that allows you to list part or all of your living space for others to rent. You can rent everything from a room in an apartment to your entire house on AirBnB. Because most of the listings are on a short-term basis, AirBnB has grown to become a popular alternative to hotels. The company itself has grown from it's founding in 2008 to a 30 billion dollar valuation in 2016 and is currently worth more than any hotel chain in the world.

One challenge that hosts looking to rent their living space face is determining the optimal nightly rent price. In many areas, renters are presented with a good selection of listings and can filter on criteria like price, number of bedrooms, room type and more. Since AirBnB is a marketplace, the amount a host can charge on a nightly basis is closely linked to the dynamics of the marketplace. Here's a screenshot of the search experience on AirBnB:

<img src='knearest/fig1.png'>

As a host, if we try to charge above market price for a living space we'd like to rent, then renters will select more affordable alternatives which are similar to ours.. If we set our nightly rent price too low, we'll miss out on potential revenue.

One strategy we could use is to:

- find a few listings that are similar to ours,
- average the listed price for the ones most similar to ours,
- set our listing price to this calculated average price.

The process of discovering patterns in existing data to make a prediction is called machine learning. In our case, we want to use data on local listings to predict the optimal price for us to set. In this mission, we'll explore a specific machine learning technique called k-nearest neighbors, which mirrors the strategy we just described. Before we dive further into machine learning and k-nearest neighbors, let's get familiar with the dataset we'll be working with.

While AirBnB doesn't release any data on the listings in their marketplace, a separate group named Inside AirBnB has extracted data on a sample of the listings for many of the major cities on the website. In this post, we'll be working with their dataset from October 3, 2015 on the listings from Washington, D.C., the capital of the United States. Here's a direct link to that dataset. Each row in the dataset is a specific listing that's available for renting on AirBnB in the Washington, D.C. area

To make the dataset less cumbersome to work with, we've removed many of the columns in the original dataset and renamed the file to dc_airbnb.csv. Here are the columns we kept:

- host_response_rate: the response rate of the host
- host_acceptance_rate: number of requests to the host that convert to rentals
- host_listings_count: number of other listings the host has
- latitude: latitude dimension of the geographic coordinates
- longitude: longitude part of the coordinates
- city: the city the living space resides
- zipcode: the zip code the living space resides
- state: the state the living space resides
- accommodates: the number of guests the rental can accommodate
- room_type: the type of living space (Private room, Shared room or Entire home/apt
- bedrooms: number of bedrooms included in the rental
- bathrooms: number of bathrooms included in the rental
- beds: number of beds included in the rental
- price: nightly price for the rental
- cleaning_fee: additional fee used for cleaning the living space after the guest leaves
- security_deposit: refundable security deposit, in case of damages
- minimum_nights: minimum number of nights a guest can stay for the rental
- maximum_nights: maximum number of nights a guest can stay for the rental
- number_of_reviews: number of reviews that previous guests have left

Let's read the dataset into Pandas and become more familiar with it.

<b>Instructions</b>
- Read dc_airbnb.csv into a Dataframe named dc_listings.
- Use the print function to display the first row in dc_listings.

In [1]:
import pandas as pd
import numpy as np

In [2]:
dc_listings = pd.read_csv('dc_airbnb.csv')
print(dc_listings.iloc[0])

host_response_rate                  92%
host_acceptance_rate                91%
host_listings_count                  26
accommodates                          4
room_type               Entire home/apt
bedrooms                              1
bathrooms                             1
beds                                  2
price                           $160.00
cleaning_fee                    $115.00
security_deposit                $100.00
minimum_nights                        1
maximum_nights                     1125
number_of_reviews                     0
latitude                          38.89
longitude                      -77.0028
city                         Washington
zipcode                           20003
state                                DC
Name: 0, dtype: object


Here's the strategy we wanted to use:

- Find a few similar listings.
- Calculate the average nightly rental price of these listings.
- Set the average price as the price for our listing.

The k-nearest neighbors algorithm is similar to this strategy. Here's an overview:

<img src='knearest/fig2.png'>

There are 2 things we need to unpack in more detail:

- the similarity metric
- how to choose the k value

In this mission, we'll define what similarity metric we're going to use. Then, we'll implement the k-nearest neighbors algorithm and use it to suggest a price for a new, unpriced listing. We'll use a k value of 5 in this mission. In later missions, we'll learn how to evaluate how good the suggested prices are, how to choose the optimal k value, and more.

The similarity metric works by comparing a fixed set of numerical features, another word for attributes, between 2 observations, or living spaces in our case. When trying to predict a continuous value, like price, the main similarity metric that's used is Euclidean distance. Here's the general formula for Euclidean distance:

$d=(q_1−p_1)^2+(q_2−p_2)2+⋯+(q_n−p_n)^2$

where $q_1$ to $q_n$ represent the feature values for one observation and $p_1$ to $p_n$ represent the feature values for the other observation. Here's a diagram that breaks down the Euclidean distance between the first 2 observations in the dataset using only the host_listings_count, accommodates, bedrooms, bathrooms, and beds columns:

<img src='knearest/fig3.png'>

In this mission, we'll use just one feature in this mission to keep things simple as you become familiar with the machine learning workflow. Since we're only using one feature, this is known as the univariate case. Here's how the formula looks like for the univariate case:

$d=(q_1−p_1)^2$

The square root and the squared power cancel and the formula simplifies to:

$d=|q_1−p_1|$

The living space that we want to rent can accommodate 3 people. Let's first calculate the distance, using just the accommodates feature, between the first living space in the dataset and our own.

<b>Instructions</b>
- Calculate the Euclidean distance between our living space, which can accommodate 3 people, and the first living space in the dc_listings Dataframe.
- Assign the result to first_distance and display the value using the print function.

In [3]:
first_distance = abs(3 - dc_listings['accommodates'].iloc[0])
print('Distance:', first_distance)

Distance: 1


The Euclidean distance between the first row in the dc_listings Dataframe and our own living space is 1. How do we know if this is high or low? If you look at the Euclidean distance equation itself, the lowest value you can achieve is 0. This happens when the value for the feature is exactly the same for both observations you're comparing. If $p_1 = q_1$, then $d = |q_1 - p_1|$, which results in $d = 0$. The closer to 0 the distance the more similar the living spaces are.

If we wanted to calculate the Euclidean distance between each living space in the dataset and a living space that accommodates 8 people, here's a preview of what that would look like.

<img src='knearest/fig4.png', height=500, width=500>

Then, we can rank the existing living spaces by ascending distance values, the proxy for similarity.

<b>Instructions</b><br>
- Calculate the distance between each value in the accommodates column from dc_listings and the value 3, which is the number of people our listing accommodates:
    - Use the apply method to calculate the absolute value between each value in accommodates and 3 and return a new Series containing the distance values.
- Assign the distance values to the distance column.
- Use the Series method value_counts and the print function to display the unique value counts for the distance column.

In [4]:
dc_listings['distance'] = dc_listings['accommodates'].apply(lambda x: abs(x - 3))

print(dc_listings['distance'].value_counts().sort_index())

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64


It looks like there are quite a few, 461 to be precise, living spaces that can accommodate 3 people just like ours. This means the 5 "nearest neighbors" we select after sorting all will have a distance value of 0. If we sort by the distance column and then just select the first 5 living spaces, we would be biasing the result to the ordering of the dataset.


    >> dc_listings[dc_listings["distance"] == 0]["accommodates"]
    26      3
    34      3
    36      3
    40      3
    44      3
    45      3
    48      3
    65      3
    66      3
    71      3
    75      3
    86      3
    ...

Let's instead randomize the ordering of the dataset and then sort the Dataframe by the distance column. This way, all of the living spaces with the same number of bedrooms will still be at the top of the Dataframe but will be in random order across the first 461 rows. We've already done the first step of setting the random seed, so we can perform answer checking on our end.

<b>Instructions</b>
- Randomize the order of the rows in dc_listings:
    - Use the np.random.permutation() function to return a NumPy array of shuffled index values.
    - Use the Dataframe method loc[] to return a new Dataframe containing the shuffled order.
    - Assign the new Dataframe back to dc_listings.
- After randomization, sort dc_listings by the distance column.
- Display the first 10 values in the price column using the print function.

In [5]:
np.random.seed(1)

shuffled_indices = np.random.permutation(len(dc_listings))
dc_listings = dc_listings.loc[shuffled_indices]
dc_listings.sort_values(by='distance', inplace=True)

dc_listings.head(10)

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
577,98%,52%,49,3,Private room,1.0,1.0,2.0,$185.00,NaN,NaN,2,14,1,38.908356,-77.028146,Washington,20005,DC,0
2166,100%,89%,2,3,Entire home/apt,1.0,1.0,1.0,$180.00,NaN,$100.00,1,14,10,38.905808,-77.000012,Washington,20002,DC,0
3631,98%,52%,49,3,Entire home/apt,1.0,1.0,2.0,$175.00,NaN,NaN,3,14,1,38.889065,-76.993576,Washington,20003,DC,0
71,100%,94%,1,3,Entire home/apt,1.0,1.0,1.0,$128.00,$40.00,NaN,1,1125,9,38.879960,-77.006491,Washington,20003,DC,0
1011,NaN,NaN,1,3,Entire home/apt,0.0,1.0,1.0,$115.00,NaN,NaN,1,1125,0,38.907382,-77.035075,Washington,20005,DC,0
380,58%,51%,480,3,Entire home/apt,NaN,1.0,1.0,$219.00,NaN,NaN,4,1125,0,38.900820,-77.052956,Washington,20037,DC,0
943,NaN,NaN,1,3,Private room,1.0,1.0,1.0,$125.00,$25.00,NaN,1,1125,3,38.913882,-77.038468,Washington,20009,DC,0
3107,NaN,NaN,1,3,Entire home/apt,1.0,1.0,1.0,$250.00,NaN,NaN,1,1125,0,38.934027,-77.035193,Washington,20010,DC,0
1499,62%,77%,1,3,Entire home/apt,0.0,1.0,2.0,$94.00,NaN,NaN,1,1125,2,38.878030,-77.019914,Washington,20024,DC,0
625,100%,0%,1,3,Entire home/apt,1.0,1.0,1.0,$150.00,NaN,NaN,1,1125,5,38.907031,-77.029510,Washington,20005,DC,0


Before we can select the 5 most similar living spaces and compute the average price, we need to clean the price column. Right now, the price column contains comma characters and dollar sign characters and is formatted as a text column instead of a numeric one. We need to remove these values and convert the entire column to the float datatype. Then, we can calculate the average price.

<b>Instructions</b><br>
- Remove the commas and dollar sign characters from the price column:
    - Use the str accessor so we can apply string methods to each value in the column followed by the string method replace to replace all comma characters with the empty character: stripped_commas = dc_listings['price'].str.replace(',', '')
    - Repeat to remove the dollar sign characters as well.
- Convert the new Series object containing the cleaned values to the float datatype and assign back to the price column in dc_listings.
- Calculate the mean of the first 5 values in the price column and assign to mean_price.
- Use the print function or the variable inspector below to display mean_price.

In [6]:
stripped = (dc_listings['price'].str.replace(',', '')
                                .str.replace('$', '')
                                .astype('float'))

dc_listings['price'] = stripped
mean_price = np.mean(dc_listings['price'].iloc[:5])
print('Mean price:', mean_price)

Mean price: 156.6


Congrats! You've just made your first prediction! Based on the average price of other listings that accommdate 3 people, we should charge 156.6 dollars per night for a guest to stay at our living space. In the next mission, we'll dive into evaluating how good of a prediction this is.

Let's write a more general function that can suggest the optimal price for other values of the accommodates column. The dc_listings Dataframe has information specific to our living space, e.g. the distance column. To save you time, we've reset the dc_listings Dataframe to a clean slate and only kept the data cleaning and randomization we did since those weren't unique to the prediction we were making for our living space.

<b>Instructions</b><br>
- Write a function named predict_price that can use the k-nearest neighbors machine learning technique to calculate the suggested price for any value for accommodates. This function should:
    - Take in a single parameter, new_listing, that describes the number of bedrooms.
    - We've added code that assigns dc_listings to a new Dataframe named temp_df. We used the pandas.DataFrame.copy() method so the underlying dataframe is assigned to temp_df, instead of just a reference to dc_listings.
    - Calculate the distance between each value in the accommodates column and the new_listing value that was passed in. Assign the resulting Series object to the distance column in temp_df.
    - Sort temp_df by the distance column and select the first 5 values in the price column. Don't randomize the ordering of temp_df.
    - Calculate the mean of these 5 values and use that as the return value for the entire predict_price function.
- Use the predict_price function to suggest a price for a living space that:
    - accommodates 1 person, assign the suggested price to acc_one.
    - accommodates 2 people, assign the suggested price to acc_two.
    - accommodates 4 people, assign the suggested price to acc_four.

In [7]:
def prepare_df(shuffle=False):

    dc_listings = pd.read_csv('dc_airbnb.csv')
    stripped_commas = dc_listings['price'].str.replace(',', '')
    stripped_dollars = stripped_commas.str.replace('$', '')
    dc_listings['price'] = stripped_dollars.astype('float')
    if shuffle:
        np.random.seed(1)
        dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]
    
    return dc_listings

In [8]:
dc_listings = prepare_df(shuffle=True)

def predict_price(new_listing):
    temp_df = dc_listings.copy()
    temp_df['distance'] = abs(new_listing - temp_df['accommodates'])
    temp_df.sort_values(by='distance', inplace=True)
    return temp_df['price'].iloc[:5].mean()

acc_one = predict_price(1)
acc_two = predict_price(2)
acc_four = predict_price(4)

print('Price for 1:', acc_one)
print('Price for 2:', acc_two)
print('Price for 4:', acc_four)

Price for 1: 68.0
Price for 2: 112.8
Price for 4: 124.8


In this mission, we explored the problem of predicting the optimal price to list an AirBnB rental for based on the price of similar listings on the site. We stepped through the entire machine learning workflow, from selecting a feature to testing the model. To explore the basics of machine learning, we limited ourselves to only using one feature (the univariate case) and a fixed k value of 5.

In the next mission, we'll learn how to evaluate a model's performance.

# Evaluating Model Performance

We now have a function that can predict the price for any living space we want to list as long as we know the number of people it can accommodate. The function we wrote represents a machine learning model, which means that it outputs a prediction based on the input to the model.

A simple way to test the quality of your model is to:

- split the dataset into 2 partitions:
    - the training set: contains the majority of the rows (75%)
    - the test set: contains the remaining minority of the rows (25%)
- use the rows in the training set to predict the price value for the rows in the test set
    - add new column named predicted_price to the test set
- compare the predicted_price values with the actual price values in the test set to see how accurate the predicted values were.

This validation process, where we use the training set to make predictions and the test set to predict values for, is known as train/test validation. Whenever you're performing machine learning, you want to perform validation of some kind to ensure that your machine learning model can make good predictions on new data. While train/test validation isn't perfect, we'll use it to understand the validation process, to select an error metric, and then we'll dive into a more robust validation process later in this course.

Let's modify the predict_price function to use only the rows in the training set, instead of the full dataset, to find the nearest neighbors, average the price values for those rows, and return the predicted price value. Then, we'll use this function to predict the price for just the rows in the test set. Once we have the predicted price values, we can compare with the true price values and start to understand the model's effectiveness in the next screen.

To start, we've gone ahead and assigned the first 75% of the rows in dc_listings to train_df and the last 25% of the rows to test_df. Here's a diagram explaining the split:

<img src='knearest/fig5.png'>

<b>Instructions</b><br>
- Within the predict_price function, change the Dataframe that temp_df is assigned to. Change it from dc_listings to train_df, so only the training set is used.
- Use the Series method apply to pass all of the values in the accommodates column from test_df through the predict_price function.
- Assign the resulting Series object to the predicted_price column in test_df.

In [9]:
dc_listings = prepare_df()

train_rows = int(0.75 * dc_listings.shape[0])
train_df = dc_listings.iloc[:train_rows]
test_df = dc_listings.iloc[train_rows:]

In [10]:
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
0,92%,91%,26,4,Entire home/apt,1.0,1.0,2.0,160.0,$115.00,$100.00,1,1125,0,38.890046,-77.002808,Washington,20003,DC
1,90%,100%,1,6,Entire home/apt,3.0,3.0,3.0,350.0,$100.00,NaN,2,30,65,38.880413,-76.990485,Washington,20003,DC
2,90%,100%,2,1,Private room,1.0,2.0,1.0,50.0,NaN,NaN,2,1125,1,38.955291,-76.986006,Hyattsville,20782,MD
3,100%,NaN,1,2,Private room,1.0,1.0,1.0,95.0,NaN,NaN,1,1125,0,38.872134,-77.019639,Washington,20024,DC
4,92%,67%,1,4,Entire home/apt,1.0,1.0,1.0,50.0,$15.00,$450.00,7,1125,0,38.996382,-77.041541,Silver Spring,20910,MD


In [11]:
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    
    return(predicted_price)

In [12]:
test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


We now need a metric that quantifies how good the predictions were on the test set. This class of metrics is called an error metric. As the name suggests, an error metric quantifies how inaccurate our predictions were from the actual values. In our case, the error metric tells us how off our predicted price values were from the actual price values for the living spaces in the test dataset.

We could start by calculating the difference between each predicted and actual value and then averaging these differences. This is referred to as mean error, but isn't an effective error metric for most cases. Mean error treats a positive difference differently than a negative difference, but we're really interested in how far off the prediction is in either the positive or negative direction. If the true price was 200 dollars and the model predicted 210 or 190 it's off by 10 dollars either way.

We can instead use the mean absolute error, where we compute the absolute value of each error before we average all the errors.

<b>Instructions</b><br>
- Use numpy.absolute() to calculate the mean absolute error between predicted_price and price.
- Assign the MAE to mae.

In [13]:
mae = np.mean(np.absolute(test_df['predicted_price'] - test_df['price']))
print('Mean Absolute Error:', mae)

Mean Absolute Error: 56.29001074113856


For many prediction tasks, we want to penalize predicted values that are further away from the actual value much more than those that are closer to the actual value.

We can instead take the mean of the squared error values, which is called the mean squared error or MSE for short. The MSE makes the gap between the predicted and actual values more clear. A prediction that's off by 100 dollars will have an error (of 10,000) that's 100 times more than a prediction that's off by only 10 dollars (which will have an error of 100).

Let's calculate the MSE value for the predictions we made on the test set.

<b>Instructions</b><br>
- Calculate the MSE value between the predicted_price and price columns and assign to mse.

In [14]:
mse = np.mean(((test_df['predicted_price'] - test_df['price']) ** 2))
rmse = np.sqrt(mse)
print('Mean Squared Error (accomodates):', mse)
print('Root Mean Squared Error (accomodates):', rmse)

Mean Squared Error (accomodates): 18646.525370569278
Root Mean Squared Error (accomodates): 136.55228072269344


While comparing MSE values helps us identify which model performs better on a relative basis, it doesn't help us understand if the performance is good enough in general. This is because the units of the MSE metric are squared (in this case, dollars squared). An MSE value of 16377.5 dollars squared doesn't give us an intuitive sense of how far off the model's predictions are systematically off from the true price value in dollars.

Root mean squared error is an error metric whose units are the base unit (in our case, dollars). RMSE for short, this error metric is calculated by taking the square root of the MSE value:

$RMSE = \sqrt(MSE)$

Since the RMSE value uses the same units as the target column, we can understand how far off in real dollars we can expect the model to perform. For example, if a model achieves an RMSE value of greater than 100, we can expect the predicted price value to be off by 100 dollars on average.

Let's calculate the RMSE value of the model we trained using the bathrooms column.

<b>Instructions</b>
- Calculate the RMSE value of the model we trained using the bathrooms column and assign it to rmse.

In [15]:
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    
    return(predicted_price)

In [16]:
test_df['predicted_price'] = test_df['bathrooms'].apply(lambda x: predict_price(x))

test_df['squared_error'] = (test_df['predicted_price'] - test_df['price']) ** (2)
mse = test_df['squared_error'].mean()
rmse = np.sqrt(mse)
print('MSE (bathrooms):', mse)
print('RMSE (bathrooms):', rmse)

MSE (bathrooms): 18405.44408163265
RMSE (bathrooms): 135.66666532952246


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The model achieved an RMSE value of approximately 135.6, which implies that we should expect for the model to be off by 135.6 dollars on average for the predicted price values. Given that most of the living spaces are listed at just a few hundred dollars, we need to reduce this error as much as possible to improve the model's usefulness.

We discussed a few different error metrics we can use to understand a model's performance. As we mentioned earlier, these individual error metrics are helpeful for comparing models. To better understand a specific model, we can compare multiple error metrics for the same model. This requires a better understanding of the mathematical properties of the error metrics.

If you look at the equation for MAE, you'll notice that a prediction that the individual errors (or differences between predicted and actual values) grow linearly. A prediction that's off by 10 dollars has a 10 times higher error than a prediction that's off by 1 dollar. If you look at the equation for RMSE, however, you'll notice that each error is squared before the square root of the sum of all the errors is taken. This means that the individual errors grows quadratically and has a different effect on the final RMSE value.

Let's look at an example using different data entirely. We've created 2 Series objects containing 2 sets of errors and assigned to errors_one and errors_two.

<b>Instructions</b><br>
- Calculate the MAE for errors_one and assign to mae_one.
- Calculate the RMSE for errors_one and assign to rmse_one.
- Calculate the MAE for errors_two and assign to mae_two.
- Calculate the RMSE for errors_two and assign to rmse_two.

In [17]:
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

mae_one = np.mean(errors_one)
rmse_one = np.sqrt(np.mean(errors_one ** 2))
mae_two = np.mean(errors_two)
rmse_two = np.sqrt(np.mean(errors_two ** 2))

print(mae_one, rmse_one)
print(mae_two, rmse_two)

7.5 7.905694150420948
62.5 235.82302686548658


While the MAE (7.5) to RMSE (7.9056941504209481) ratio was about 1:1 for the first list of errors, the MAE (62.5) to RMSE (235.82302686548658) ratio was closer to 1:4 for the second list of errors. In general, we should expect that the RMSE value be much less than the MAE value because we're taking the square root of the squared errors. The only difference between the 2 sets of errors is the extreme 1000 value in errors_two instead of 10. When we're working with larger data sets, we can't inspect each value to understand if there's one or some outliers or if all of the errors are systematically higher. Looking at the ratio of MAE to RMSE can help us understand if there are large but infrequent errors. You can read more about comparing MAE and RMSE in this wonderful post.

In this mission, we learned how to test our machine learning models using basic cross validation and different metrics. In the next 2 missions, we'll explore how adding more features to the machine learning model and selecting a more optimal k value can help improve the model's performance.

# Multivariate K-Nearest Neighbors

In the last mission, we explored how to use a simple k-nearest neighbors machine learning model that used just one feature, or attribute, of the listing to predict the rent price. We first relied on the accommodates column, which describes the number of people a living space can comfortably accommodate. Then, we switched to the bathrooms column and observed an improvement in accuracy. While these were good features to become familiar with the basics of machine learning, it's clear that using just a single feature to compare listings doesn't reflect the reality of the market. An apartment that can accommodate 4 guests in a popular part of Washington D.C. will rent for much higher than one that can accommodate 4 guests in a crime ridden area.

There are 2 ways we can tweak the model to try to improve the accuracy (decrease the RMSE during validation):

- increase the number of attributes the model uses to calculate similarity when ranking the closest neighbors
- increase k, the number of nearby neighbors the model uses when computing the prediction

In this mission, we'll focus on increasing the number of attributes the model uses. When selecting more attributes to use in the model, we need to watch out for columns that don't work well with the distance equation. This includes columns containing:

- non-numerical values (e.g. city or state)
    - Euclidean distance equation expects numerical values
- missing values
    - distance equation expects a value for each observation and attribute
- non-ordinal values (e.g. latitude or longitude)
    - ranking by Euclidean distance doesn't make sense if all attributes aren't ordinal

In the following code screen, we've read the dc_airbnb.csv dataset from the last mission into pandas and brought over the data cleaning changes we made. Let's first look at the first row's values to identify any columns containing non-numerical or non-ordinal values. In the next screen, we'll drop those columns and then look for missing values in each of the remaining columns.

<b>Instructions</b><br>
- Use the DataFrame.info() method to return the number of non-null values in each column

In [18]:
dc_listings = prepare_df(shuffle=True)

dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3723 entries, 574 to 1061
Data columns (total 19 columns):
host_response_rate      3289 non-null object
host_acceptance_rate    3109 non-null object
host_listings_count     3723 non-null int64
accommodates            3723 non-null int64
room_type               3723 non-null object
bedrooms                3702 non-null float64
bathrooms               3696 non-null float64
beds                    3712 non-null float64
price                   3723 non-null float64
cleaning_fee            2335 non-null object
security_deposit        1426 non-null object
minimum_nights          3723 non-null int64
maximum_nights          3723 non-null int64
number_of_reviews       3723 non-null int64
latitude                3723 non-null float64
longitude               3723 non-null float64
city                    3723 non-null object
zipcode                 3714 non-null object
state                   3723 non-null object
dtypes: float64(6), int64(5), objec

The following columns contain non-numerical values:

- room_type: e.g. Private room
- city: e.g. Washington
- state: e.g. DC

while these columns contain numerical but non-ordinal values:

- latitude: e.g. 38.913458
- longitude: e.g. -77.031
- zipcode: e.g. 20009

Geographic values like these aren't ordinal, because a smaller numerical value doesn't directly correspond to a smaller value in a meaningful way. For example, the zip code 20009 isn't smaller or larger than the zip code 75023 and instead both are unique, identifier values. Latitude and longitude value pairs describe a point on a geographic coordinate system and different equations are used in those cases (e.g. haversine).

While we could convert the host_response_rate and host_acceptance_rate columns to be numerical (right now they're object data types and contain the % sign), these columns describe the host and not the living space itself. Since a host could have many living spaces and we don't have enough information to uniquely group living spaces to the hosts themselves, let's avoid using any columns that don't directly describe the living space or the listing itself:

- host_response_rate
- host_acceptance_rate
- host_listings_count

Let's remove these 9 columns from the Dataframe.

<b>Instructions</b><br>
- Remove the 9 columns we discussed above from dc_listings:
    - 3 containing non-numerical values
    - 3 containing numerical but non-ordinal values
    - 3 describing the host instead of the living space itself

In [19]:
cols_to_drop = ['room_type', 'city', 'state', 
                'latitude', 'longitude', 'zipcode', 
                'host_response_rate', 'host_acceptance_rate', 'host_listings_count']

dc_listings.drop(cols_to_drop, axis=1, inplace=True)

In [20]:
dc_listings.isnull().sum()

accommodates            0
bedrooms               21
bathrooms              27
beds                   11
price                   0
cleaning_fee         1388
security_deposit     2297
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

Of the remaining columns, 3 columns have a few missing values (less than 1% of the total number of rows):

- bedrooms
- bathrooms
- beds

Since the number of rows containing missing values for one of these 3 columns is low, we can select and remove those rows without losing much information. There are also 2 columns have a large number of missing values:

- cleaning_fee - 37.3% of the rows
- security_deposit - 61.7% of the rows

and we can't handle these easily. We can't just remove the rows containing missing values for these 2 columns because we'd miss out on the majority of the observations in the dataset. Instead, let's remove these 2 columns entirely from consideration.

<b>Instructions</b><br>
- Drop the cleaning_fee and security_deposit columns from dc_listings.
- Then, remove all rows that contain a missing value for the bedrooms, bathrooms, or beds column from dc_listings.
    - You can accomplish this by using the Dataframe method dropna() and setting the axis parameter to 0.
    - Since only the bedrooms, bathrooms, and beds columns contain any missing values, rows containing missing values in these columns will be removed.
- Display the null value counts for the updated dc_listings Dataframe to confirm that there are no missing values left.

In [21]:
dc_listings.drop(['cleaning_fee', 'security_deposit'], axis=1, inplace=True)
dc_listings.dropna(axis=0, inplace=True)
dc_listings.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64

In [22]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
574,2,1.0,1.0,1.0,125.0,1,4,149
1593,2,1.0,1.5,1.0,85.0,1,30,49
3091,1,1.0,0.5,1.0,50.0,1,1125,1
420,2,1.0,1.0,1.0,209.0,4,730,2
808,12,5.0,2.0,5.0,215.0,2,1825,34


You may have noticed that while the accommodates, bedrooms, bathrooms, beds, and minimum_nights columns hover between 0 and 12 (at least in the first few rows), the values in the maximum_nights and number_of_reviews columns span much larger ranges. For example, the maximum_nights column has values as low as 4 and high as 1825, in the first few rows itself. If we use these 2 columns as part of a k-nearest neighbors model, these attributes could end up having an outsized effect on the distance calculations because of the largeness of the values.

For example, 2 living spaces could be identical across every attribute but be vastly different just on the maximum_nights column. If one listing had a maximum_nights value of 1825 and the other a maximum_nights value of 4, because of the way Euclidean distance is calculated, these listings would be considered very far apart because of the outsized effect the largeness of the values had on the overall Euclidean distance. To prevent any single column from having too much of an impact on the distance, we can normalize all of the columns to have a mean of 0 and a standard deviation of 1.

Normalizing the values in each columns to the standard normal distribution (mean of 0, standard deviation of 1) preserves the distribution of the values in each column while aligning the scales. To normalize the values in a column to the standard normal distribution, you need to:

from each value, subtract the mean of the column
divide each value by the standard deviation of the column
Here's the mathematical formula describing the transformation that needs to be applied for all values in a column:

$x = (x - \mu) / \sigma$

where  is a value in a specific column,  is the mean of all the values in the column, and  is the standard deviation of all the values in the column. Here's what the corresponding code, using pandas, looks like:


    # Subtract each value in the column by the mean.
    first_transform = dc_listings['maximum_nights'] - dc_listings['maximum_nights'].mean()
    # Divide each value in the column by the standard deviation.
    normalized_col = first_transform / first_transform.std()

To apply this transformation across all of the columns in a Dataframe, you can use the corresponding Dataframe methods mean() and std():

    normalized_listings = (dc_listings - dc_listings.mean()) / (dc_listings.std())

These methods were written with mass column transformation in mind and when you call mean() or std(), the appropriate column means and column standard deviations are used for each value in the Dataframe. Let's now normalize all of the feature columns in dc_listings.

<b>Instructions</b><br>
- Normalize all of the feature columns in dc_listings and assign the new Dataframe containing just the normalized feature columns to normalized_listings.
- Add the price column from dc_listings to normalized_listings.
- Display the first 3 rows in normalized_listings.

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
scaler = StandardScaler()
normalized_listings = scaler.fit_transform(dc_listings)
normalized_listings = pd.DataFrame(normalized_listings, index=dc_listings.index, columns=dc_listings.columns)
normalized_listings['price'] = dc_listings['price']

In [25]:
normalized_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
574,-0.596625,-0.249501,-0.439211,-0.546933,125.0,-0.341421,-0.016606,4.580274
1593,-0.596625,-0.249501,0.412979,-0.546933,85.0,-0.341421,-0.016606,1.159433
3091,-1.095648,-0.249501,-1.291402,-0.546933,50.0,-0.341421,-0.016575,-0.482571
420,-0.596625,-0.249501,-0.439211,-0.546933,209.0,0.487701,-0.016586,-0.448362
808,4.393602,4.508517,1.265170,2.830341,215.0,-0.065047,-0.016555,0.646307


In the last mission, we trained 2 univariate k-nearest neighbors models. The first one used the accommodates attribute while the second one used the bathrooms attribute. Let's now train a model that uses both attributes when determining how similar 2 living spaces are. Let's refer to the Euclidean distance equation again to see what the distance calculation using 2 attributes would look like:

$d = \sqrt(q_1 - p_1)^2 + (q_2 - p_2)^2 + ... + (q_n - p_n)^2$

Since we're using 2 attributes, the distance calculation would look like:

$d = \sqrt(accomodates_1 - accomodates_1)^2 + (bathrooms_2 - bathrooms_2)^2$

To find the distance between 2 living spaces, we need to calculate the squared difference between both accommodates values, the squared difference between both bathrooms values, add them together, and then take the square root of the resulting sum. Here's what the Euclidean distance between the first 2 rows in normalized_listings looks like:

<img src='knearest/fig6.png', height=300, width=600>

So far, we've been calculating Euclidean distance ourselves by writing the logic for the equation ourselves. We can instead use the distance.euclidean() function from scipy.spatial, which takes in 2 vectors as the parameters and calculates the Euclidean distance between them. The euclidean() function expects:

- both of the vectors to be represented using a list-like object (Python list, NumPy array, or pandas Series)
- both of the vectors must be 1-dimensional and have the same number of elements

Here's a simple example:

    from scipy.spatial import distance
    first_listing = [-0.596544, -0.439151]
    second_listing = [-0.596544, 0.412923]
    dist = distance.euclidean(first_listing, second_listing)

Let's use the euclidean() function to calculate the Euclidean distance between 2 rows in our dataset to practice.

<b>Instructions</b><br>
- Calculate the Euclidean distance using only the accommodates and bathrooms features between the first row and fifth row in normalized_listings using the distance.euclidean() function.
- Assign the distance value to first_fifth_distance and display using the print function.

In [26]:
from scipy.spatial import distance

In [27]:
first = normalized_listings[['bathrooms', 'accommodates']].iloc[0]
fifth = normalized_listings[['bathrooms', 'accommodates']].iloc[4]

first_fifth_difference = distance.euclidean(first, fifth)
print('Distance:', first_fifth_difference)

Distance: 5.273261405868558


So far, we've been writing functions from scratch to train the k-nearest neighbor models. While this is helpful deliberate practice to understand how the mechanics work, you can be more productive and iterate quicker by using a library that handles most of the implementation. In this screen, we'll learn about the scikit-learn library, which is the most popular machine learning in Python. Scikit-learn contains functions for all of the major machine learning algorithms and a simple, unified workflow. Both of these properties allow data scientists to be incredibly productive when training and testing different models on a new dataset.

The scikit-learn workflow consists of 4 main steps:

- instantiate the specific machine learning model you want to use
- fit the model to the training data
- use the model to make predictions
- evaluate the accuracy of the predictions

We'll focus on the first 3 steps in this screen and the next screen. Each model in scikit-learn is implemented as a separate class and the first step is to identify the class we want to create an instance of. In our case, we want to use the KNeighborsRegressor class.
Any model that helps us predict numerical values, like listing price in our case, is known as a regression model. The other main class of machine learning models is called classification, where we're trying to predict a label from a fixed set of labels (e.g. blood type or gender). The word regressor from the class name KNeighborsRegressor refers to the regression model class that we just discussed.

Scikit-learn uses a similar object-oriented style to Matplotlib and you need to instantiate an empty model first by calling the constructor:

    from sklearn.neighbors import KNeighborsRegressor
    knn = KNeighborsRegressor()

If you refer to the documentation, you'll notice that by default:

- n_neighbors: the number of neighbors, is set to 5
- algorithm: for computing nearest neighbors, is set to auto
- p: set to 2, corresponding to Euclidean distance

Let's set the algorithm parameter to brute and leave the n_neighbors value as 5, which matches the implementation we wrote in the last mission. If we leave the algorithm parameter set to the default value of auto, scikit-learn will try to use tree-based optimizations to improve performance (which are outside of the scope of this mission):

    knn = KNeighborsRegressor(algorithm='brute')
    
Now, we can fit the model to the data using the fit method. For all models, the fit method takes in 2 required parameters:

- matrix-like object, containing the feature columns we want to use from the training set.
- list-like object, containing correct target values.

Matrix-like object means that the method is flexible in the input and either a Dataframe or a NumPy 2D array of values is accepted. This means you can select the columns you want to use from the Dataframe and use that as the first parameter to the fit method.

If you recall from earlier in the mission, all of the following are acceptable list-like objects:

- NumPy array
- Python list
- pandas Series object (e.g. when selecting a column)

You can select the target column from the Dataframe and use that as the second parameter to the fit method:


    # Split full dataset into train and test sets.
    train_df = normalized_listings.iloc[0:2792]
    test_df = normalized_listings.iloc[2792:]
    # Matrix-like object, containing just the 2 columns of interest from training set.
    train_features = train_df[['accommodates', 'bathrooms']]
    # List-like object, containing just the target column, `price`.
    train_target = normalized_listings['price']
    # Pass everything into the fit method.
    knn.fit(train_features, train_target)

When the fit() method is called, scikit-learn stores the training data we specified within the KNearestNeighbors instance (knn). If you try passing in data containing missing values or non-numerical values into the fit method, scikit-learn will return an error. Scikit-learn contains many such features that help prevent us from making common mistakes.

Now that we specified the training data we want used to make predictions, we can use the predict method to make predictions on the test set. The predict method has only one required parameter:

matrix-like object, containing the feature columns from the dataset we want to make predictions on
The number of feature columns you use during both training and testing need to match or scikit-learn will return an error:

    predictions = knn.predict(test_df[['accommodates', 'bathrooms']])

The predict() method returns a NumPy array containing the predicted price values for the test set. You now have everything you need to practice the entire scikit-learn workflow.

<b>Instructions</b><br>
- Create an instance of the KNeighborsRegressor class with the following parameters:
    - n_neighbors: 5
    - algorithm: brute
- Use the fit method to specify the data we want the k-nearest neighbor model to use. Use the following parameters:
    - training data, feature columns: just the accommodates and bathrooms columns, in that order, from train_df.
    - training data, target column: the price column from train_df.
- Call the predict method to make predictions on:
    - the accommodates and bathrooms columns from test_df
    - assign the resulting NumPy array of predicted price values to predictions.

In [28]:
from sklearn.neighbors import KNeighborsRegressor

In [29]:
train_rows = int(0.75 * dc_listings.shape[0])
train_df = normalized_listings.iloc[:train_rows]
test_df = normalized_listings.iloc[train_rows:]

In [30]:
knr = KNeighborsRegressor(n_neighbors=5, algorithm='brute')
knr.fit(train_df[['accommodates', 'bathrooms']], train_df['price'])
predictions = knr.predict(test_df[['accommodates', 'bathrooms']])

Earlier in this mission, we calculated the MSE and RMSE values using the pandas arithmetic operators to compare each predicted value with the actual value from the price column of our test set. Alternatively, we can instead use the sklearn.metrics.mean_squared_error function(). Once you become familiar with the different machine learning concepts, unifying your workflow using scikit-learn helps save you a lot of time and avoid mistakes.

The mean_squared_error() function takes in 2 inputs:

    -list-like object, representing the true values
    -list-like object, representing the predicted values using the model

For this function, we won't show any sample code and will leave it to you to understand the function from the documentation itself to calculate the MSE and RMSE values for the predictions we just made.

<b>Instructions</b><br>
- Use the mean_squared_error function to calculate the MSE value for the predictions we made in the previous screen.
- Assign the MSE value to two_features_mse.
- Calculate the RMSE value by taking the square root of the MSE value and assign to two_features_rmse.
- Display both of these error scores using the print function.

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
mse = mean_squared_error(test_df['price'], predictions)
rmse = np.sqrt(mse)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)

Mean Squared Error: 14840.477167755991
Root Mean Squared Error: 121.8214971495425


Here's a table comparing the MSE and RMSE values for the 2 univariate models from the last mission and the multivariate model we just trained:

<img src='knearest/fig7.jpg', height=162, width=370>

As you can tell, the model we trained using both features ended up performing better (lower error score) than either of the univariate models from the last mission. Let's now train a model using the following 4 features:

- accommodates
- bedrooms
- bathrooms
- number_of_reviews

Scikit-learn makes it incredibly easy to swap the columns used during training and testing. We're going to leave this for you as a challenge to train and test a k-nearest neighbors model using these columns instead. Use the code you wrote in the last screen as a guide.

<b>Instructions</b><br>
- Create a new instance of the KNeighborsRegressor class with the following parameters:
    - n_neighbors: 5
    - algorithm: brute
- Fit a model that uses the following columns from our training set (train_df):
    - accommodates
    - bedrooms
    - bathrooms
    - number_of_reviews
- Use the model to make predictions on the test set (test_df) using the same columns. Assign the NumPy array of predictions to four_predictions.
- Use the mean_squared_error() function to calculate the MSE value for these predictions by comparing four_predictions with the price column from test_df. Assign the computed MSE value to four_mse.
- Calculate the RMSE value and assign to four_rmse.
- Display four_mse and four_rmse using the print function.

In [33]:
knr = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

knr.fit(train_df[['accommodates', 'bathrooms', 'bedrooms', 'number_of_reviews']], train_df['price'])
predictions = knr.predict(test_df[['accommodates', 'bathrooms', 'bedrooms', 'number_of_reviews']])

mse = mean_squared_error(test_df['price'], predictions)
rmse = np.sqrt(mse)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)

Mean Squared Error: 13602.571764705881
Root Mean Squared Error: 116.63006372589308


So far so good! As we increased the features the model used, we observed lower MSE and RMSE values:

<img src='knearest/fig8.jpg', height=218, width=600>

Let's take this to the extreme and use all of the potential features. We should expect the error scores to decrease since so far adding more features has helped do so.

<b>Instructions</b><br>
- Use all of the columns, except for the price column, to train a k-nearest neighbors model using the same parameters for the KNeighborsRegressor class as the ones from the last few screens.
- Use the model to make predictions on the test set and assign the resulting NumPy array of predictions to all_features_predictions.
- Calculate the MSE and RMSE values and assign to all_features_mse and all_features_rmse accordingly.
- Use the print function to display both error scores.

In [34]:
features = train_df.columns.drop('price')

knr = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

knr.fit(train_df[features], train_df['price'])
predictions = knr.predict(test_df[features])

mse = mean_squared_error(test_df['price'], predictions)
rmse = np.sqrt(mse)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)

Mean Squared Error: 15041.373594771241
Root Mean Squared Error: 122.64327782137609


Interestingly enough, the RMSE value actually increased to 124.3 when we used all of the features available to us. This means that selecting the right features is important and that using more features doesn't automatically improve prediction accuracy. We should re-phrase the lever we mentioned earlier from:

- increase the number of attributes the model uses to calculate similarity when ranking the closest neighbors

to:

- select the relevant attributes the model uses to calculate similarity when ranking the closest neighbors

The process of selecting features to use in a model is known as feature selection.

In this mission, we prepared the data to be able to use more features, trained a few models using multiple features, and evaluated the different performance tradeoffs. We explored how using more features doesn't always improve the accuracy of a k-nearest neighbors model. In the next mission, we'll explore another knob for tuning k-nearest neighbor models - the k value.

# Hyperparameter Optimization

In the last mission, we focused on increasing the number of attributes the model uses. We saw how, in general, adding more attributes generally lowered the error of the model. This is because the model is able to do a better job identifying the living spaces from the training set that are the most similar to the ones from the test set. However, we also observed how using all of the available features didn't actually improve the model's accuracy automatically and that some of the features were probably not relevant for similarity ranking. We learned that selecting relevant features was the right lever when improving a model's accuracy, not just increasing the features used in the absolute.

In this mission, we'll focus on the impact of increasing k, the number of nearby neighbors the model uses to make predictions. We exported both the training (train_df) and test sets (test_df) from the last missions to CSV files, dc_airbnb_train.csv and dc_airbnb_test.csv respectively. Let's read both these CSV's into Dataframes.

<b>Instructions</b><br>
- Read dc_airbnb_train.csv into a Dataframe and assign to train_df.
- Read dc_airbnb_test.csv into a Dataframe and assign to test_df.

In [35]:
# Use the same dfs as above (numbers will be different)

When we vary the features that are used in the model, we're affecting the data that the model uses. On the other hand, varying the k value affects the behavior of the model independently of the actual data that's used when making predictions. In other words, we're impacting how the model performs without trying to change the data that's used.

Values that affect the behavior and performance of a model that are unrelated to the data that's used are referred to as hyperparameters. The process of finding the optimal hyperparameter value is known as hyperparameter optimization. A simple but common hyperparameter optimization technique is known as grid search, which involves:

- selecting a subset of the possible hyperparameter values,
- training a model using each of these hyperparameter values,
- evaluating each model's performance,
- selecting the hyperparameter value that resulted in the lowest error value.

Grid search essentially boils down to evaluating the model performance at different k values and selecting the k value that resulted in the lowest error. While grid search can take a long time when working with large datasets, the data we're working with in this mission is small and this process is relatively quick.

Let's confirm that grid search will work quickly for the dataset we're working with by first observing how the model performance changes as we increase the k value from 1 to 5. If you recall, we set 5 as the k value for the last 2 missions. Let's use the features from the last mission that resulted in the best model accuracy:

- accommodates
- bedrooms
- bathrooms
- number_of_reviews

<b>Instructions</b><br>
- Create a list containing the integer values 1, 2, 3, 4, and 5, in that order, and assign to hyper_params.
- Create an empty list and assign to mse_values.
- Use a for loop to iterate over hyper_params and in each iteration:
    - Instantiate a KNeighborsRegressor object with the following parameters:
        - n_neighbors: the current value for the iterator variable,
        - algorithm: brute
    - Fit the instantiated k-nearest neighbors model to the following columns from train_df:
        - accommodates
        - bedrooms
        - bathrooms
        - number_of_reviews
    - Use the trained model to make predictions on the same columns from test_df and assign to predictions.
    - Use the mean_squared_error function to calculate the MSE value between predictions and the price column from test_df.
    - Append the MSE value to mse_values.
- Display mse_values using the print() function.

In [36]:
features = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']
hyper_params = [1, 2, 3, 4, 5]
mse_values = []

for k in hyper_params:
    knn = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    knn.fit(train_df[features], train_df['price'])
    prediction = knn.predict(test_df[features])
    
    mse_values.append(mean_squared_error(test_df['price'], prediction))

print(mse_values)

[25759.187363834422, 14771.49537037037, 14270.824013556039, 16476.91489651416, 14088.38779956427]


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
hyperparameters = {'n_neighbors': list(range(1, 100)),
                  'algorithm': ['brute', 'auto']}

knn = KNeighborsRegressor()
grid = GridSearchCV(knn, param_grid=hyperparameters)
grid.fit(train_df[features], train_df['price'])

print(grid.best_estimator_)

Since our dataset is small and scikit-learn has been developed with performance in mind, the code ran quickly. As we increased the k value from 1 to 5, the MSE value fell from approximately 26364 to approximately 14090:

Let's expand grid search all the way to a k value of 20. While 20 may seem like an arbitrary ending point for our grid search, we can always expand the values we try if we're unconvinced that the lowest MSE value is associated with one of the hyperparamter values we tried so far.

<b>Instructions</b><br>
- Change the list of hyperparameter values, hyper_params, so it ranges from 1 to 20.
- Create an empty list and assign to mse_values.
- Use a for loop to iterate over hyper_params and in each iteration:
    - Instantiate a KNeighborsRegressor object with the following parameters:
        - n_neighbors: the current value for the iterator variable,
        - algorithm: brute
    - Fit the instantiated k-nearest neighbors model to the following columns from train_df:
        - accommodates
        - bedrooms
        - bathrooms
        - number_of_reviews
    - Use the trained model to make predictions on the same columns from test_df and assign to predictions.
    - Use the mean_squared_error function to calculate the MSE value between predictions and the price column from test_df.
    - Append the MSE value to mse_values.
- Display mse_values using the print() function.

In [ ]:
hyper_params = list(range(1, 21))
mse_values = []

for k in hyper_params:
    knn = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    knn.fit(train_df[features], train_df['price'])
    prediction = knn.predict(test_df[features])
    
    mse_values.append(mean_squared_error(test_df['price'], prediction))

print(mse_values)

As we increased the k value from 1 to 6, the MSE value decreased from approximately 26364 to approximately 13657. However, as we increased the k value from 7 to 20, the MSE value didn't decrease further but instead hovered between approximately 14288 and 14870. This means that the optimal k value is 6, since it resulted in the lowest MSE value.

This pattern is something you'll notice while performing grid search across other models as well. As you increase k at first, the error rate decreases until a certain point, but then rebounds and increases again. Let's confirm this behavior visually using a scatter plot.

<b>Instructions</b>
- Use the scatter() method from matplotlib.pyplot to generate a line plot with:
    - hyper_params on the x-axis,
    - mse_values on the y-axis.
- Use plt.show() to display the line plot.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
plt.scatter(hyper_params, mse_values)

From the scatter plot, you can tell that the lowest MSE value was achieved at the k value of 6. As we increased k past 6, the MSE actually increased and hovered but never decreased below 13657 (the approximate MSE value when k was 6).

Since varying the k value decreased the MSE value for this model, you may be wondering if repeating the grid search process for one of the models from the last mission that performed poorly when we fixed k to 5 would result in a lower MSE value. Let's try it out!

<b>Instructions</b>
- Use a for loop to iterate over hyper_params and in each iteration:
    - Instantiate a KNeighborsRegressor object with the following parameters:
        - n_neighbors: the current value for the iterator variable,
        - algorithm: brute
    - Fit the instantiated k-nearest neighbors model to all of the columns, except for the price column, from train_df
    - Use the trained model to make predictions on the same columns from test_df and assign to predictions.
    - Use the mean_squared_error function to calculate the MSE value between predictions and the price column from test_df.
    - Append the MSE value to mse_values.
- Use the scatter() method from matplotlib.pyplot to generate a line plot with:
    - hyper_params on the x-axis,
    - mse_values on the y-axis.
- Use plt.show() to display the line plot.

In [ ]:
features = train_df.columns.drop('price')
hyper_params = list(range(1, 21))
mse_values = []

for k in hyper_params:
    knn = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    knn.fit(train_df[features], train_df['price'])
    prediction = knn.predict(test_df[features])
    
    mse_values.append(mean_squared_error(test_df['price'], prediction))

plt.scatter(hyper_params, mse_values)

You may have noticed that the general workflow for finding the best model is:

- select relevant features to use for predicting the target column.
- use grid search to find the optimal hyperparameter value for the selected features.
- evaluate the model's accuracy and repeat the process.

Let's now practice this workflow.

<b>Instructions</b><br>
- While using only the accommodates and bathrooms columns:
    - Train a model for each k value between 1 and 20 using the training data.
    - Use each model to make predictions on the test set (using just the accommodates and bathrooms columns).
    - Calculate each model's MSE value by comparing each set of predictions to the true price values.
    - Find the k value that obtained the lowest MSE value.
    - Create a dictionary named two_hyp_mse that contains 1 key-value pair:
        - key: k value that resulted in lowest MSE value.
        - value: corresponding MSE value.
- Repeat this process while using only the accommodates, bathrooms, and bedrooms columns:
    - Create a dictionary named three_hyp_mse that contains 1 key-value pair:
        - key: k value that resulted in lowest MSE value.
        - value: corresponding MSE value.
- Display both two_hyp_mse and three_hyp_mse using the print() function.

In [ ]:
two_features = ['accommodates', 'bathrooms']
three_features = ['accommodates', 'bathrooms', 'bedrooms']
hyper_params = range(1, 21)
two_mse_values = []
three_mse_values = []
two_hyp_mse = {}
three_hyp_mse = {}

for k in hyper_params:
    knn = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    knn.fit(train_df[two_features], train_df['price'])
    prediction = knn.predict(test_df[two_features])
    
    two_mse_values.append(mean_squared_error(test_df['price'], prediction))
    
for k in hyper_params:
    knn = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    knn.fit(train_df[three_features], train_df['price'])
    prediction = knn.predict(test_df[three_features])
    
    three_mse_values.append(mean_squared_error(test_df['price'], prediction))
    
two_hyp_mse = {two_mse_values.index(min(two_mse_values)) + 1: min(two_mse_values)}
three_hyp_mse = {three_mse_values.index(min(three_mse_values)) + 1: min(three_mse_values)}

print('MSE (two features):', two_hyp_mse)
print('MSE (three features):', three_hyp_mse)

The first model, which used the accommodates and bathrooms columns, was able to achieve an MSE value of approximately 14790. The second model, which added the bedrooms column, was able to achieve an MSE value of approximately 13522.9, which is even lower than the lowest MSE value we achieved using the best model from the last mission (which used the accommodates, bedrooms, bathrooms, and number_of_reviews columns). Hopefully this demonstrates that using just one lever to find the best model isn't enough and you really want to use both levers in conjunction.

In this mission, we learned about hyperparameter optimization and the workflow of finding the optimal model to make predictions. Next in this course is a challenge, where you'll practice the concepts you've learned so far on a completely new dataset.

# Cross Validation

In an earlier mission, we learned about train/test validation, a simple technique for testing a machine learning model's accuracy on new data that the model wasn't trained on. In this mission, we'll focus on more robust techniques.

To start, we'll focus on the holdout validation technique, which involves:
- splitting the full dataset into 2 partitions:
    - a training set
    - a test set
- training the model on the training set,
- using the trained model to predict labels on the test set,
- computing an error metric to understand the model's effectiveness,
- switch the training and test sets and repeat,
- average the errors.
- In holdout validation, we usually use a 50/50 split instead of the 75/25 split from train/test validation. This way, we remove number of observations as a potential source of variation in our model performance.

<img src='knearest/fig9.png', height=500, width=500>

Let's start by splitting the data set into 2 nearly equivalent halves.

<b>Instructions</b><br>
- Use the numpy.random.permutation() function to shuffle the ordering of the rows in dc_listings.
- Select the first 1862 rows and assign to split_one.
- Select the remaining 1861 rows and assign to split_two.

In [ ]:
dc_listings = prepare_df(shuffle=True)
split_one = dc_listings.iloc[:1862]
split_two = dc_listings.iloc[1862:]

Now that we've split our data set into 2 dataframes, let's:

- train a k-nearest neighbors model on the first half,
- test this model on the second half,
- train a k-nearest neighbors model on the second half,
- test this model on the first half.

<b>Instructions</b><br>
- Train a k-nearest neighbors model using the default algorithm (auto) and the default number of neighbors (5) that:
    - Uses the accommodates column from train_one for training and
    - Tests it on test_one.
- Assign the resulting RMSE value to iteration_one_rmse.
- Train a k-nearest neighbors model using the default algorithm (auto) and the default number of neighbors (5) that:
    - Uses the accommodates column from train_two for training and
    - Tests it on test_two.
- Assign the resulting RMSE value to iteration_two_rmse.
- Use numpy.mean() to calculate the average of the 2 RMSE values and assign to avg_rmse.

In [ ]:
knn = KNeighborsRegressor()
knn.fit(split_one[['accommodates']], split_one['price'])
prediction = knn.predict(split_two[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(split_two['price'], prediction))

knn = KNeighborsRegressor()
knn.fit(split_two[['accommodates']], split_two['price'])
prediction = knn.predict(split_one[['accommodates']])
iteration_two_rmse = np.sqrt(mean_squared_error(split_one['price'], prediction))

avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])
print('Average RMSE:', avg_rmse)

If we average the two RMSE values from the last step, we get an RMSE value of approximately 128.96. Holdout validation is actually a specific example of a larger class of validation techniques called k-fold cross-validation. While holdout validation is better than train/test validation because the model isn't repeatedly biased towards a specific subset of the data, both models that are trained only use half the available data. K-fold cross validation, on the other hand, takes advantage of a larger proportion of the data during training while still rotating through different subsets of the data to avoid the issues of train/test validation.

Here's the algorithm from k-fold cross validation:

- splitting the full dataset into k equal length partitions,
    - selecting k-1 partitions as the training set and
    - selecting the remaining partition as the test set
- training the model on the training set,
- using the trained model to predict labels on the test fold,
- computing the test fold's error metric,
- repeating all of the above steps k-1 times, until each partition has been used as the test set for an iteration,
- calculating the mean of the k error values.

Holdout validation is essentially a version of k-fold cross validation when k is equal to 2. Generally, 5 or 10 folds is used for k-fold cross-validation. Here's a diagram describing each iteration of 5-fold cross validation:

<img src='knearest/fig10.png', height=300, width=500>

As you increase the number the folds, the number of observations in each fold decreases and the variance of the fold-by-fold errors increases. Let's start by manually partitioning the data set into 5 folds. Instead of splitting into 5 dataframes, let's add a column that specifies which fold the row belongs to. This way, we can easily select

<b>Instructions</b>
- Add a new column to dc_listings named fold that contains the fold number each row belongs to:
    - Fold 1 should have rows from index 0 to 744, including both of those rows.
    - Fold 2 should have rows from index 744 to 1488, including both of those rows.
    - Fold 3 should have rows from index 1488 to 2232, including both of those rows.
    - Fold 4 should have rows from index 2232 to 2976, including both of those rows.
    - Fold 5 should have rows from index 2976 to 3723, including both of these rows.
- Display the unique value counts for the fold column to confirm that each fold has roughly the same number of elements.

In [ ]:
dc_listings.at[dc_listings.index[0:744], "fold"] = 1
dc_listings.at[dc_listings.index[744:1488], "fold"] = 2
dc_listings.at[dc_listings.index[1488:2232], "fold"] = 3
dc_listings.at[dc_listings.index[2232:2976], "fold"] = 4
dc_listings.at[dc_listings.index[2976:3723], "fold"] = 5

dc_listings['fold'].value_counts()

Let's start by performing the first iteration of k-fold cross validation on a simple, univariate model.

<b>Instructions</b><br>
- Train a k-nearest neighbors model using the accommodates column as the sole feature from folds 2 to 5 as the training set.
- Use the model to make predictions on the test set (accommodates column from fold 1) and assign the predicted labels to labels.
- Calculate the RMSE value by comparing the price column with the predicted labels.
- Assign the RMSE value to iteration_one_rmse.

In [ ]:
train_df = dc_listings[dc_listings['fold'] != 1]
test_df = dc_listings[dc_listings['fold'] == 1]

knn = KNeighborsRegressor()
knn.fit(train_df[['accommodates']], train_df['price'])
prediction = knn.predict(test_df[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(test_df['price'], prediction))
print('RMSE:', iteration_one_rmse)

From the first iteration, we achieved an RMSE value of 105.1. Let's calculate the RMSE values for the remaining iterations. To make the iteration process easier, let's wrap the code we wrote in the previous screen in a function.

<b>Instructions</b>
- Write a function named train_and_validate that takes in a dataframe as the first parameter (df) and a list of fold values (1 to 5 in our case) as the second parameter (folds). This function should:
    - Train n models (where n is number of folds) and perform k-fold cross validation (using n folds). Use the default k value for the KNeighborsRegressor class.
    - Return a list of RMSE values, where the first element is the RMSE for when fold 1 was the test set, the second element is the RMSE for when fold 2 was the test set, and so on.
- Use the train_and_validate function to return the list of RMSE values for the dc_listings Dataframe and assign to rmses.
- Calculate the mean of these values and assign to avg_rmse.
- Display both rmses and avg_rmse.

In [ ]:
def train_and_validate(df, fold_ids):
    
    rmses = []
    
    for fold_id in fold_ids:
        train_df = df[df['fold'] != fold_id]
        test_df = df[df['fold'] == fold_id]
        
        knn = KNeighborsRegressor()
        knn.fit(train_df[['accommodates']], train_df['price'])
        prediction = knn.predict(test_df[['accommodates']])
        rmse = np.sqrt(mean_squared_error(test_df['price'], prediction))
        rmses.append(rmse)
    
    return rmses

In [ ]:
fold_ids = [1, 2, 3, 4, 5]
rmses = train_and_validate(dc_listings, fold_ids)
avg_rmse = np.mean(rmses)
print(rmses, avg_rmse)

While the average RMSE value was approximately 135.67, the RMSE values ranged from 105.06 all the way to 171.40. This large amount of variability between the RMSE values means that we're either using a poor model or a poor evaluation criteria (or a bit of both!). By implementing your own k-fold cross-validation function, you hopefully acquired a good understanding of the inner workings of the technique. The function we wrote, however, has many limitations. If we want to now change the number of folds we want to use, we need to make the function more general so it can also handle randomizing the ordering of the rows in the dataframe and splitting into folds.

In machine learning, we're interested in building a good model and accurately understanding how well it will perform. To build a better k-nearest neighbors model, we can change the features it uses or tweak the number of neighbors (a hyperparameter). To accurately understand a model's performance, we can perform k-fold cross validation and select the proper number of folds. We've learned how scikit-learn makes it easy for us to quickly experiment with these different knobs when it comes to building a better model. Let's now dive into how we can use scikit-learn to handle cross-validation as well.

First, we instantiate an instance of the KFold class from sklearn.model_selection:

    from sklearn.model_selection import KFold
    kf = KFold(n_splits, shuffle=False, random_state=None)
where:

- n_splits is the number of folds you want to use,
- shuffle is used to toggle shuffling of the ordering of the observations in the dataset,
- random_state is used to specify the random seed value if shuffle is set to True.

You'll notice here that no parameters depend on the data set at all. This is because the KFold class returns an iterator object which we use in conjunction with the cross_val_score() function, also from sklearn.model_selection. Together, these 2 functions allow us to compactly train and test using k-fold cross validation:

Here are the relevant parameters for the cross_val_score function:

    from sklearn.model_selection import cross_val_score
    cross_val_score(estimator, X, Y, scoring=None, cv=None)

where:

- estimator is a sklearn model that implements the fit method (e.g. instance of KNeighborsRegressor),
- X is the list or 2D array containing the features you want to train on,
- y is a list containing the values you want to predict (target column),
- scoring is a string describing the scoring criteria (list of accepted values here).
- cv describes the number of folds. Here are some examples of accepted values:
    - an instance of the KFold class,
    - an integer representing the number of folds.

Depending on the scoring criteria you specify, either a single total value is returned one value for each fold. Here's the general workflow for performing k-fold cross-validation using the classes we just described:

- instantiate the scikit-learn model class you want to fit,
- instantiate the KFold class and using the parameters to specify the k-fold cross-validation attributes you want,
- use the cross_val_score() function to return the scoring metric you're interested in.

<b>Instructions</b>
- Create a new instance of the KFold class with the following properties:
    - 5 folds,
    - shuffle set to True,
    - random seed set to 1 (so we can answer check using the same seed),
    - assigned to the variable kf.
- Create a new instance of the KNeighborsRegressor class and assign to knn.
- Use the cross_val_score() function to perform k-fold cross-validation:
    - using the KNeighborsRegressor instance knn,
    - using the accommodates column for training,
    - using the price column as the target column,
    - returning an array of MSE values (one value for each fold).
- Assign the resulting list of MSE values to mses. Then, take the absolute value followed by the square root of each mse value. Then, calculate the average of the resulting RMSE values and assign to avg_rmse.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
dc_listings = prepare_df(shuffle=True)

cols_to_drop = ['room_type', 'city', 'state', 
                'latitude', 'longitude', 'zipcode', 
                'host_response_rate', 'host_acceptance_rate', 'host_listings_count']
dc_listings.drop(cols_to_drop, axis=1, inplace=True)
dc_listings.drop(['cleaning_fee', 'security_deposit'], axis=1, inplace=True)
dc_listings.dropna(axis=0, inplace=True)

scaler = StandardScaler()
normalized_listings = scaler.fit_transform(dc_listings)
normalized_listings = pd.DataFrame(normalized_listings, index=dc_listings.index, columns=dc_listings.columns)
normalized_listings['price'] = dc_listings['price']
dc_listings = normalized_listings

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()
mses = cross_val_score(knn, dc_listings[['accommodates']], dc_listings['price'], scoring='neg_mean_squared_error', cv=kf)
avg_rmse = np.sqrt(np.mean(np.abs(mses)))
print('Average RMSE (k=5):', avg_rmse)

Choosing the right k value when performing k-fold cross validation is more of an art and less of a science. As we discussed earlier in the mission, a k value of 2 is really just holdout validation. On the other end, setting k equal to n (the number of observations in the data set) is known as leave-one-out cross validation, or LOOCV for short. Through lots of trial and error, data scientists have converged on 10 as the standard k value.

In the following code block, we display the results of varying k from 3 to 23. For each k value, we calculate and display the average RMSE value across all of the folds and the standard deviation of the RMSE values. Across the many different k values, it seems like the average RMSE value is around 128. You'll notice that the standard deviation of the RMSE increases from approximately 1.1 to 37.3 as we increase the number the folds.

In [ ]:
num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

print('ONE FEATURE MODEL')
for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print('k =', str(fold), "\tAverage RMSE:", str(avg_rmse), "\tStd RMSE:", str(std_rmse))
    
print('\nTWO FEATURE MODEL')
for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates", 'bedrooms']], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print('k =', str(fold), "\tAverage RMSE:", str(avg_rmse), "\tStd RMSE:", str(std_rmse))
    
print('\nTHREE FEATURE MODEL')
for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates", 'bedrooms', 'bathrooms']], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print('k =', str(fold), "\tAverage RMSE:", str(avg_rmse), "\tStd RMSE:", str(std_rmse))
    
print('\nFOUR FEATURE MODEL')
for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates", 'bedrooms', 'bathrooms', 'number_of_reviews']], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print('k =', str(fold), "\tAverage RMSE:", str(avg_rmse), "\tStd RMSE:", str(std_rmse))

So far, we've been working under the assumption that a lower RMSE always means that a model is more accurate. This isn't the complete picture, unfortunately. A model has two sources of error, bias and variance.

Bias describes error that results in bad assumptions about the learning algorithm. For example, assuming that only one feature, like a car's weight, relates to a car's fuel efficiency will lead you to fit a simple, univariate regression model that will result in high bias. The error rate will be high since a car's fuel efficiency is affected by many other factors besides just its weight.

Variance describes error that occurs because of the variability of a model's predicted values. If we were given a dataset with 1000 features on each car and used every single feature to train an incredibly complicated multivariate regression model, we will have low bias but high variance. In an ideal world, we want low bias and low variance but in reality, there's always a tradeoff.

The standard deviation of the RMSE values can be a proxy for a model's variance while the average RMSE is a proxy for a model's bias. Bias and variance are the 2 observable sources of error in a model that we can indirectly control.

<img src='knearest/fig11.png'>

While k-nearest neighbors can make predictions, it isn't a mathematical model. A mathematical model is usually an equation that can exist without the original data, which isn't true with k-nearest neighbors. In the next two courses, we'll learn about a mathematical model called linear regression. We'll explore the bias-variance tradeoff in greater depth in these next 2 courses because of its importance when working with mathematical models in particular.

In [ ]:
dc_listings['number_of_reviews'].isnull().sum()